In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [3]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [4]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [5]:
from sklearn.metrics import f1_score

In [6]:
X_train = train_df["title"].values
X_test = test_df["title"].values
y_train = train_df["target"].astype(int).values

# Simple baseline

In [7]:
y_pred = [int("порно" in text) for text in X_train]

In [8]:
f1_score(y_train, y_pred)

0.31242758551206484

### Submit

In [9]:
test_df["target"] = [("порно" in text) for text in X_test]

test_df[["id", "target"]].to_csv("simple_baseline.csv", index=False)

!cat simple_baseline.csv | head

id,target
135309,False
135310,False
135311,False
135312,False
135313,False
135314,False
135315,False
135316,False
135317,False
cat: ошибка записи: Обрыв канала


### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip

# ML baseline

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier

vectorizer = CountVectorizer()

model = DecisionTreeClassifier(max_depth=40)

In [11]:
X_train_vectorized = vectorizer.fit_transform(X_train)

In [12]:
X_train_vectorized

<135309x189852 sparse matrix of type '<class 'numpy.int64'>'
	with 1202659 stored elements in Compressed Sparse Row format>

In [14]:
feature_names = np.array(vectorizer.get_feature_names())

In [15]:
id_ = 42

print(X_train[id_])

x_vector = X_train_vectorized.getrow(id_).toarray()[0]

[feature for feature in feature_names[x_vector > 0]]

Презентация на тему "Приближенное значение. Абсолютная и относительная погрешнос


['абсолютная',
 'значение',
 'на',
 'относительная',
 'погрешнос',
 'презентация',
 'приближенное',
 'тему']

In [13]:
%%time

model.fit(
    X_train_vectorized,
    y_train
)

y_pred = model.predict(
    X_train_vectorized
)

CPU times: user 6.92 s, sys: 27.9 ms, total: 6.94 s
Wall time: 6.2 s


In [17]:
f1_score(y_train, y_pred)

0.31242758551206484

In [13]:
kf = KFold(n_splits=2,shuffle=True, random_state= 42)
kf.split(X_train_vectorized) 
for train_index, test_index in kf.split(X_train_vectorized):
    X_train_train, X_test_test = X_train_vectorized[train_index], X_train_vectorized[test_index]
    Y_train_train, Y_test_test = y_train[train_index], y_train[test_index]
    
    
X_train_train, X_test_test , Y_train_train, Y_test_test = train_test_split(X_train_vectorized, y_train,
                                                                           random_state = 42, test_size = 0.2)
    

from sklearn.model_selection import GridSearchCV

''''
clf = DecisionTreeClassifier(max_depth=30, 
                             min_samples_leaf=1, 
                             min_samples_split=2, 
                             criterion='gini')

parameters = {'max_depth': list(range(60,150,3)), 
              'min_samples_leaf':list(range(2,40,2)),
              'min_samples_split':list(range(2,40,2))}

gs = GridSearchCV(clf,                 # Classifier object to optimize
                  parameters,          # Grid of the hyperparameters
                  scoring='accuracy',  # Claasification quality metric to optimize
                  cv=3)                # Number of folds in KFolds cross-validation (CV)


gs.fit(X_train_vectorized, y_train)
gs.best_params_*/
'''

"'\nclf = DecisionTreeClassifier(max_depth=30, \n                             min_samples_leaf=1, \n                             min_samples_split=2, \n                             criterion='gini')\n\nparameters = {'max_depth': list(range(60,150,3)), \n              'min_samples_leaf':list(range(2,40,2)),\n              'min_samples_split':list(range(2,40,2))}\n\ngs = GridSearchCV(clf,                 # Classifier object to optimize\n                  parameters,          # Grid of the hyperparameters\n                  scoring='accuracy',  # Claasification quality metric to optimize\n                  cv=3)                # Number of folds in KFolds cross-validation (CV)\n\n\ngs.fit(X_train_vectorized, y_train)\ngs.best_params_*/\n"

In [17]:
penalty = ['l1', 'l2', 'elasticnet']
tol = np.arange(1e-10, 1e-3, 0.000199998)
C = np.arange(1.0, 20, 2)
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

index = 1
maxf1 = 0
maxpen = ''
maxtol = 0
maxC = 0
maxsolver = ''

for i1 in penalty:
    for i2 in tol:
        for i3 in C:
            for i4 in solver:
                print()
                try:
                    f1 = CalculateRegrassion(i1,i2,i3,i4)
                    print(f1)
                    if f1 > maxf1:
                        maxf1 = f1
                        maxpen = i1
                        maxtol = i2
                        maxC = i3
                        maxsolver = i4
                except:
                    print("Errro")
                print(index)
                print()
                index += 1

In [14]:
def CalculateRegrassion(penalty, tol, C, solver):
    print('Param: penalty = {penalty}, tol = {tol}, C = {C}, solver = {solver}'.format(
        penalty= penalty, tol = tol, C = C, solver = solver))
    start = time.time()
    logreg = LogisticRegression(penalty= penalty, tol = tol, C = C, solver = solver) 
    logreg.fit(X_train_train, Y_train_train) 
    y_pred = logreg.predict(X_test_test)
    finish = time.time()
    print("time: {}".format(finish - start))
    return f1_score(Y_test_test, y_pred)

In [55]:
print(CalculateRegrassion('l2',1e-10, 16, 'lbfgs'))

Param: penalty = l2, tol = 1e-10, C = 16, solver = lbfgs
time: 2.7867603302001953
0.9564950038431975


In [59]:
model = LogisticRegression(penalty='l2',tol=1e-10, C=16, solver= 'lbfgs') 
model.fit(X_train_vectorized, y_train) 
y_pred = model.predict(X_train_vectorized)
print(f1_score(y_train, y_pred))

0.9938653996511697


### Submit

In [60]:
X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)